In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [2]:
cmm_data = pd.read_csv('/home/ruggiec/Downloads/pharmacy_tx.csv')

#cmm_data_no_dups = cmm_data.drop_duplicates()

cmm_train, cmm_test = train_test_split(cmm_data, shuffle=True, 
                             random_state= 614, test_size=0.2)

In [6]:
def is_it_generic(text):
    if text.split()[0] == 'generic':
        return 1
    else:
        return 0
    
def drug_name(text):
    return text.split()[1]

def pharm_split(text):
    return int(text.split('#')[1])

In [4]:
cmm_train['tx_date'] = pd.to_datetime(cmm_train.tx_date)
cmm_train['month'] = cmm_train.tx_date.dt.month
cmm_train['day_of_year'] = cmm_train.tx_date.dt.dayofyear

In [7]:
cmm_train['generic'] = cmm_train.drug.apply(is_it_generic)
cmm_train['name'] = cmm_train['drug'].apply(drug_name)
cmm_train['private'] = cmm_train.group.isna().astype(int)
cmm_train['popularity'] = cmm_train.groupby('name')[['tx_date']].transform('count')
cmm_train['rejected'] = cmm_train['rejected'].astype(int)
cmm_train['rejected_count'] = cmm_train.groupby('name')['rejected'].transform('sum')
cmm_train['pharm_num'] = cmm_train.pharmacy.apply(pharm_split)

In [9]:
cmm_train['binpcn'] = cmm_train.bin.astype(str).fillna('') + cmm_train.pcn.fillna('')
cmm_train['bingroup'] = cmm_train.bin.astype(str).fillna('') + cmm_train.group.fillna('')
cmm_train['binpcngroup'] = cmm_train.bin.astype(str).fillna('') + cmm_train.pcn.fillna('') + cmm_train.group.fillna('')

In [21]:
print('There are', cmm_train.binpcngroup.nunique(), 
      'separate insurance plans in the data set.')
print('There are', cmm_train.drug.nunique(), 
      'drugs in the data set.')
print('Leading to', cmm_train.binpcngroup.nunique()*cmm_train.drug.nunique(), 
      'combinations of drug and insurance plan.')

There are 63 separate insurance plans in the data set.
There are 114 drugs in the data set.
Leading to 7182 combinations of drug and insurance plan.


In [36]:
cmm_train.binpcngroup.sample(1).values[0]

'7257009C5MOR3S2QKZ0OFNWS6X'

In [50]:
cmm_train[cmm_train.binpcngroup == 
          cmm_train.binpcngroup.sample(1).values[0]].groupby('drug')['patient_pay'].unique()

drug
branded antimab            [42.85, 20.32, 21.06, 20.22, 21.85, 21.36, 21....
branded bovirol                                                      [12.64]
branded brede                                                         [12.8]
branded choxestamenium     [150.93, 150.37, 150.24, 0.0, 150.92, 150.71, ...
branded cibroniudosin                                                [11.15]
                                                 ...                        
generic tanoclolol                                                     [6.5]
generic thiostasteglume    [14.32, 15.4, 14.57, 13.3, 14.94, 14.65, 15.99...
generic todiadianic        [48.84, 49.68, 0.0, 47.68, 48.61, 48.34, 49.35...
generic tovane                                                        [9.53]
generic vocopirin          [17.58, 12.55, 12.56, 12.9, 11.13, 11.65, 11.8...
Name: patient_pay, Length: 114, dtype: object

In [56]:
cmm_train[cmm_train.binpcngroup == 
          cmm_train.binpcngroup.sample(1).values[0]].groupby('drug')['patient_pay'].std()

drug
branded antimab             5.181009
branded bovirol             0.000000
branded brede               0.000000
branded choxestamenium     66.193669
branded cibroniudosin       0.000000
                             ...    
generic tanoclolol          0.000000
generic thiostasteglume     2.306000
generic todiadianic         8.528204
generic tovane              0.000000
generic vocopirin           1.481350
Name: patient_pay, Length: 114, dtype: float64